In [117]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from fake_headers import Headers
import re

In [118]:
def get_soup(url):
     '''
        Функция передает html-код

        Args:
            headers: генерация случайных браузеров для подключения
        Returns:
            soup: cоздается объект BeautifulSoup, HTML-данные передаются конструктору
            Fals: ошибка, в случае если ответ не 200
    '''
    try:
        headers = Headers(headers = True).generate()
        response = requests.get(url, headers = headers)
        response.raise_for_status() 
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    except requests.RequestException:
        return False

In [119]:
def write_down(weather):
    '''
        Функция добавляет в tsv файл собранные данные
        Если файла не существует, создается новый

        Args:
            weather.tsv: имя tsv файла, в который требуется добавить данные
            weather: данные, которые запишутся в файл
    '''
    df = pd.DataFrame(weather)
    df.to_csv('weather.tsv', sep='\t', index=False)
    print(df)

In [120]:
def parse():
    '''
    Собираем данные с помощью парсинга. 

    Args:
        cities: список, который содержит все коды для ссылок на страницы городов
        data: собирается словарь с данными о матчах
        all_data: возвращает список из команды, игрока, его поизиции и роста
'''
    all_data=[]
    cities = ['bournemouth','leeds','newcastle-upon-tyne','leicester','brighton',
          'southampton','wolverhampton','bradford','nottingham','watford',
         'burnley', 'norwich','liverpool', 'manchester', 'london', 'birmingham']
    for c in range(len(cities)):
        url = f'https://pogoda.365c.ru/united-kingdom/{cities[c]}/po_mesyacam'
        soup = get_soup(url)
        city = soup.find(class_='active')
        month = soup.find_all(class_='d-name')
        month = month[1:]
        temp_day = soup.find_all(class_='d-temp')
        rain = soup.find_all(class_='d-temp2')
        for i in range(len(month)):
            data = {'Город': city.text, 'Месяц': i+1, 'Название месяца': month[i].text, 
                    'Дневная температура' : temp_day[i].text, 'Дождь': rain[i].text[0],
                    'Осадки': rain[i].text.split(' ')[-1]}
            all_data.append(data)
    return all_data

In [121]:
if __name__ == '__main__':
    weather = parse()
    write_down(weather)
   
    

         Город  Месяц Название месяца Дневная температура Дождь    Осадки
0      Борнмут      1          Январь               +10°C     7  (89.5мм)
1      Борнмут      2         Февраль             +10.8°C     6  (65.4мм)
2      Борнмут      3            Март             +11.5°C     3  (33.0мм)
3      Борнмут      4          Апрель               +13°C     5  (50.8мм)
4      Борнмут      5             Май             +15.5°C     6  (48.5мм)
..         ...    ...             ...                 ...   ...       ...
187  Бирмингем      8          Август             +19.5°C     5  (82.0мм)
188  Бирмингем      9        Сентябрь             +20.8°C     3  (49.2мм)
189  Бирмингем     10         Октябрь             +15.2°C     6  (58.6мм)
190  Бирмингем     11          Ноябрь             +10.2°C     2  (45.9мм)
191  Бирмингем     12         Декабрь                +8°C     3  (66.2мм)

[192 rows x 6 columns]
